In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler

In [2]:
train=pd.read_csv('./Dataset/Undersampling_0.33_train.csv',index_col=False, encoding='euc-kr')
test=pd.read_csv('./Dataset/Undersampling_0.33_test.csv',index_col=False,encoding='euc-kr')

In [3]:
X_train_int=train[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름',
'자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률']]

X_test_int=test[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름',
'자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률']]

In [4]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [5]:
X_train_sc = X_train
X_test_sc = X_test

In [6]:
X_train_sum =X_train_sc
X_test_sum =X_test_sc

In [7]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from catboost import CatBoostClassifier
import numpy as np

def perform_catboost_grid_search(X_train, y_train, k_fold=5):
    # Stratified k-fold cross validation setup
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # CatBoost hyperparameter candidate list setup
    param_grid = {
        'depth': [3, 5, 7],
        'iterations': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'loss_function': ['Logloss'],
        'random_state': [0]
    }

    # Initialize a CatBoost classifier
    model = CatBoostClassifier(verbose=0)

    # Grid search setup
    grid_search = GridSearchCV(model, param_grid, scoring='f1', cv=cv, verbose=1, n_jobs=-1)

    # Fit the model and tune
    grid_search.fit(X_train, y_train)

    # Print the best hyperparameters
    print("Best Hyperparameters:", grid_search.best_params_)

    # Calculate the average evaluation metric
    mean_f1_score = np.mean(grid_search.cv_results_['mean_test_score'])
    print("Mean F1 Score:", mean_f1_score)

    return grid_search.best_params_, mean_f1_score


In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from catboost import CatBoostClassifier
import numpy as np

def evaluate_catboost_with_best_params(X_train, y_train, X_test, y_test, best_params, k_fold=5):
    # Stratified k-fold cross validation setup
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # Initialize a CatBoost model
    model = CatBoostClassifier(**best_params, verbose=0)

    # Lists to save the evaluation metrics for each fold
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # Fit the model
        model.fit(X_train_fold, y_train_fold)

        # Get the predicted probabilities on the test data
        probabilities = model.predict_proba(X_test_fold)

        # Adjust the predicted classes with a threshold of 0.5
        threshold = 0.5
        predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)

        # Calculate the evaluation metrics
        accuracy = accuracy_score(y_test_fold, predicted_classes)
        precision = precision_score(y_test_fold, predicted_classes)
        recall = recall_score(y_test_fold, predicted_classes)
        f1 = f1_score(y_test_fold, predicted_classes)
        conf_matrix = confusion_matrix(y_test_fold, predicted_classes)

        # Append the evaluation metrics for each fold to the lists
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # Save the model with the best F1 score
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # Perform the final prediction with the model with the best F1 score
    probabilities_final = best_model.predict_proba(X_test)
    y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)

    # Calculate the evaluation metrics
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list

In [9]:
best_params, mean_f1_score = perform_catboost_grid_search(X_train, y_train, k_fold=5)
evaluate_catboost_with_best_params(X_train_int, y_train, X_test_int, y_test, best_params, k_fold=5)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
4 fits failed out of a total of 135.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\catboost\core.py", line 5131, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_

Best Hyperparameters: {'depth': 7, 'iterations': 300, 'learning_rate': 0.2, 'loss_function': 'Logloss', 'random_state': 0}
Mean F1 Score: nan


C:\Users\dgh06\AppData\Local\Temp\ipykernel_28116\340735650.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)


Fold 1
Accuracy: 0.8356164383561644
Precision: 0.75
Recall: 0.5
F1 score: 0.6
Confusion Matrix:
[[52  3]
 [ 9  9]]
------------------------------


...

Fold 3
Accuracy: 0.7916666666666666
Precision: 0.6363636363636364
Recall: 0.3888888888888889
F1 score: 0.4827586206896552
Confusion Matrix:
[[50  4]
 [11  7]]
------------------------------


C:\Users\dgh06\AppData\Local\Temp\ipykernel_28116\340735650.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)


Fold 4
Accuracy: 0.8611111111111112
Precision: 0.7222222222222222
Recall: 0.7222222222222222
F1 score: 0.7222222222222222
Confusion Matrix:
[[49  5]
 [ 5 13]]
------------------------------
Fold 5
Accuracy: 0.8333333333333334
Precision: 0.7142857142857143
Recall: 0.5555555555555556
F1 score: 0.6250000000000001
Confusion Matrix:
[[50  4]
 [ 8 10]]
------------------------------
Final Test Results
Accuracy: 0.7870370370370371
Precision: 0.5833333333333334
Recall: 0.5185185185185185
F1 score: 0.5490196078431373
Confusion Matrix:
[[71 10]
 [13 14]]


C:\Users\dgh06\AppData\Local\Temp\ipykernel_28116\340735650.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)
C:\Users\dgh06\AppData\Local\Temp\ipykernel_28116\340735650.py:67: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

([0.8356164383561644,
  0.8082191780821918,
  0.7916666666666666,
  0.8611111111111112,
  0.8333333333333334],
 [0.75,
  0.6428571428571429,
  0.6363636363636364,
  0.7222222222222222,
  0.7142857142857143],
 [0.5, 0.5, 0.3888888888888889, 0.7222222222222222, 0.5555555555555556],
 [0.6,
  0.5625000000000001,
  0.4827586206896552,
  0.7222222222222222,
  0.6250000000000001])